In [883]:
import pandas as pd
import seaborn as sns
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import text
import urllib
import pyodbc
import os
from urllib.parse import quote_plus
from itertools import zip_longest
import re
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

In [962]:
driver = os.getenv()
server = os.getenv()
database = os.getenv()
usuario = os.getenv()
password = os.getenv()

def get_connection() -> URL:
    connection_string = """{};SERVER={};DATABASE={};UID={};PWD={}""".format(driver,server,database,usuario,password)
    url_db = quote_plus(connection_string)
    connection_url = f'mssql+pyodbc:///?odbc_connect=+{url_db}'
    return create_engine(connection_url,fast_executemany=True)

In [963]:
def vendas_hausz():
    lista_dicts = []
    engine = get_connection()
    with engine.connect() as conn:
        query_etapas_flexy = (text("""
           SELECT brand.Marca,iflexy.[SKU],basico.NomeProduto,iflexy.[CodigoPedido]
            ,iflexy.[Quantidade]
            ,iflexy.[QtdCaixa],iflexy.[PrecoUnitario],pflexy.StatusPedido
            ,format(iflexy.[DataInserido], 'd', 'pt-BR') as datapedido

            FROM [HauszMapa].[Pedidos].[ItensFlexy]as iflexy
            join [HauszMapa].[Produtos].[ProdutoBasico] as basico
            on basico.SKU = iflexy.[SKU]
            join Produtos.Marca as brand
            on brand.IdMarca = basico.IdMarca
            join [HauszMapa].[Pedidos].[PedidoFlexy] as pflexy
            on pflexy.CodigoPedido = iflexy.CodigoPedido

            where format(iflexy.[DataInserido], 'd', 'pt-BR') like '%/2022%' """))

        execqueryetapas = conn.execute(query_etapas_flexy).all()
        for exc in execqueryetapas:
            dict_items = {}
            for keys, values in exc.items():
                dict_items[keys] = values
            lista_dicts.append(dict_items)
    
    return lista_dicts

pedidos = vendas_hausz()

C:\Users\Crawler\AppData\Local\Temp\ipykernel_19840\1017627963.py:24: SADeprecationWarning: The LegacyRow.items() method is deprecated and will be removed in a future release.  Use the Row._mapping attribute, i.e., 'row._mapping.items()'. (deprecated since: 1.4)
  for keys, values in exc.items():


In [ ]:
def select_wms_rejeicoes():
    lista_dicts = []
    engine = get_connection()
    with engine.connect() as conn:
        query_etapas_flexy = (text("""
            select * from wms.T_Rejeicoes """))

        execqueryetapas = conn.execute(query_etapas_flexy).all()
        for exc in execqueryetapas:
            dict_items = {}
            for keys, values in exc.items():
                dict_items[keys] = values
            lista_dicts.append(dict_items)
    
    return lista_dicts

logrejeicao = select_wms_rejeicoes()


In [ ]:

def etapas_flexy():
    lista_dicts = []
    engine = get_connection()
    with engine.connect() as conn:
        query_etapas_flexy = (text("""
            SELECT DISTINCT [IdEtapa],[NomeEtapa]
            FROM [HauszMapa].[Pedidos].[EtapaFlexy] """))

        execqueryetapas = conn.execute(query_etapas_flexy).all()
        for exc in execqueryetapas:
            dict_items = {}
            for keys, values in exc.items():
                dict_items[keys] = values
            lista_dicts.append(dict_items)
    
    return lista_dicts

dicts = etapas_flexy()

In [ ]:
def select_log_wms_pedidos():
    lista_dicts = []
    engine = get_connection()
    with engine.connect() as conn:
        query_log = (text("""
                SELECT  lgpedido.[IdLog],lgpedido.[DeIdEtapaFlexy],lgpedido.[ParaIdEtapaFlexy]
                ,lgpedido.[CodigoPedido],pflexy.DataInserido as datainserido
                ,pflexy.DataInseridoOmie as datainseridoomie
                ,pflexy.IdEtapaFlexy,etflexy.NomeEtapa,etflexy.NomeEtapaFlexy
                ,pflexy.PrevisaoEntrega as previsaodeentrega
                ,pflexy.StatusPedido,lgpedido.[EnviadoWpp]
                ,lgpedido.[DataAtualizacao] as dataatualizacao
                ,lgpedido.[DePrazo] ,lgpedido.[ParaPrazo]
                ,lgpedido.[bitSplit],lgpedido.[TipoAlteracao],lgpedido.[IdUsuarioAlteracao]

                FROM [HauszMapa].[Pedidos].[LogPedidos] as lgpedido

                join [HauszMapa].[Pedidos].[PedidoFlexy] as pflexy
                on pflexy.CodigoPedido = lgpedido.CodigoPedido

                join [HauszMapa].[Pedidos].[EtapaFlexy] as etflexy
                on etflexy.IdEtapa = pflexy.IdEtapaFlexy"""))
            
        excquerylog = conn.execute(query_log).all()
        for exc in excquerylog:
            dict_items = {}
            for keys, values in exc.items():
                
                dict_items[keys] = values
            lista_dicts.append(dict_items)
    return lista_dicts

log_pedido = select_log_wms_pedidos()

In [967]:
def retorna_dataatual():
    data = datetime.today().strftime('%Y-%m-%d')

    return data

data_atual =  retorna_dataatual()


In [968]:
log_pedido = select_log_wms_pedidos()
dflogpedido = pd.DataFrame(log_pedido)

dfrejeicaowms = pd.DataFrame(logrejeicao)



C:\Users\Crawler\AppData\Local\Temp\ipykernel_19840\2047943344.py:27: SADeprecationWarning: The LegacyRow.items() method is deprecated and will be removed in a future release.  Use the Row._mapping attribute, i.e., 'row._mapping.items()'. (deprecated since: 1.4)
  for keys, values in exc.items():


In [980]:
#read files
dfwms1 = pd.read_excel('D:\\estatisticas_hausz\\arquivosexcelcorpem\\WD6part1.xlsx',skiprows=[0,1,2,3,4,5])
dfwms2 = pd.read_excel('D:\\estatisticas_hausz\\arquivosexcelcorpem\\WD6part2.xlsx',skiprows=[0,1,2,3,4,5])
dfwxd1 =  pd.read_excel('D:\\estatisticas_hausz\\arquivosexcelcorpem\\WXDpart1.xlsx',skiprows=[0,1,2,3,4,5])
dfwxd2 =  pd.read_excel('D:\\estatisticas_hausz\\arquivosexcelcorpem\\WXDpart2.xlsx',skiprows=[0,1,2,3,4,5])

In [1085]:
dfwxd1['UNIDADE_CD'] = 'Hausz-SP'
dfwxd1[['Dt. Inclusão','Dt. Mov.','Dt. Reserva','Dt. Conf. Sep.'
    ,'Dt. Embarque','No. D.P.','No. Cli.','Obs. Resumida','No. Ped. Cli.'
    ,'No. N.F.','Sit. Fase','Transp. (Ped.)','Destinatário','Doca','Vols.', 'UNIDADE_CD']]

dfwxd2['UNIDADE_CD'] = 'Hausz-SC'
dfwxd2[['Dt. Inclusão','Dt. Mov.','Dt. Reserva','Dt. Conf. Sep.'
    ,'Dt. Embarque','No. D.P.','No. Cli.','Obs. Resumida','No. Ped. Cli.'
    ,'No. N.F.','Sit. Fase','Transp. (Ped.)','Destinatário','Doca','Vols.', 'UNIDADE_CD']]

unificalocwms = pd.concat([dfwxd1, dfwxd2])
unificalocwms.columns

unificalocwms.rename(columns={'Dt. Inclusão':'DATA_INCLUSAO_PEDIDO','Dt. Mov.':'DATA_MOVIMENTO'
            ,'Dt. Reserva':'DATA_RESERVA','Dt. Conf. Sep.':'DATA_CONFERENCIA_SEPARACAO'
            ,'Dt. N.F.':'DATA_NOTA_FISCAL','Dt. Embarque':'DATA_EMBARQUE'
            ,'No. D.P.':'NUMERO_DP','No. Cli.':'NUMERO_CLI','Obs. Resumida':'REFERENCIA_PEDIDO_WMS'
            ,'No. Ped. Cli.':'REFERENCIA_PEDIDO_CLI','No. N.F.':'NUMERO_PEDIDO_NF'
            ,'Sit. Fase':'FASE_PEDIDO_WMS','Prior.':'PRIORIDADE','Transp. (Ped.)':'TRANSPORTADORA_PEDIDO'
            ,'Destinatário':'DESTINATARIO','Doca':'DOCA','Vols.':'VOLUMES'}, inplace=True)


unificalocwms.loc[:,['DATA_INCLUSAO_PEDIDO','DATA_MOVIMENTO','DATA_RESERVA','DATA_CONFERENCIA_SEPARACAO'
        ,'DATA_NOTA_FISCAL','DATA_EMBARQUE']] = unificalocwms.loc[:,['DATA_INCLUSAO_PEDIDO','DATA_MOVIMENTO','DATA_RESERVA'
        ,'DATA_CONFERENCIA_SEPARACAO','DATA_NOTA_FISCAL','DATA_EMBARQUE']].apply(pd.to_datetime, errors='coerce',dayfirst=True)





In [ ]:
#filtrando data atual dataframe  wms.T_Rejeicoes
dflogpedido = dflogpedido[dflogpedido.ParaIdEtapaFlexy==6]

dfrejeicaowms['Data'] = dfrejeicaowms['Data'].astype(str)
dfrejeicaowmsatual= dfrejeicaowms[dfrejeicaowms['Data'].str.contains(data_atual)]


#filtrando data atual dataframe LogPedidos
dflogpedido['dataatualizacao'] = dflogpedido['dataatualizacao'].astype(str)
dflogpedidoatual = dflogpedido[dflogpedido['dataatualizacao'].str.contains(data_atual)]

dflogpedidoatual.loc[:,'dataatualizacao'] = dflogpedidoatual.loc[:,'dataatualizacao'].max()

#seleciona o valor maior/converte em formato data/remov duplicados mantendo apenas o valor maior
dfrejeicaowmsatual.loc[:,'Data'] = pd.to_datetime(dfrejeicaowmsatual.loc[:,'Data'])
#dfrejeicaowmsatual.groupby('Data', group_keys=False).apply(lambda x: x.loc[x.Data.idxmax()])

#seleciona o valor maior/converte em formato data/remov duplicados mantendo apenas o valor maior
dflogpedidoatual.loc[:,'dataatualizacao'] = pd.to_datetime(dflogpedidoatual.loc[:,'dataatualizacao'])


dflogpedidoatual.groupby('dataatualizacao', group_keys=False).apply(lambda x: x.loc[x.dataatualizacao.idxmin()])

dflogpedidoatual

In [ ]:
dfrejeicaowmsatual['RejeicaoId'] = dfrejeicaowmsatual['RejeicaoId'].astype(str)
'''
dfrejeicaowmsatual['RejeicaoId'] = dfrejeicaowmsatual['RejeicaoId'].astype(int)
tipos = {0:'Integrou',6:'SemEstoque',3:'Erro'}


dfrejeicaowmsatual['StatusRejeicao'] = dfrejeicaowmsatual.RejeicaoId.map(tipos)

dfrejeicaowmsatual = dfrejeicaowmsatual[dfrejeicaowmsatual == 'Integrou']

freq = dfrejeicaowmsatual.loc[:,'StatusRejeicao'].value_counts(normalize=True) * 100

'''

In [ ]:
import numpy as np

dfrejeicaowmsatual.loc[:,'CodigoPedido'] = dfrejeicaowmsatual.loc[:,'CodigoPedido'].astype(str)


dflogpedidoatual.loc[:,'CodigoPedido'] = dflogpedidoatual.loc[:,'CodigoPedido'].astype(str)
'''
dfrejeicaowmsatual = dfrejeicaowmsatual.loc[:,['CodigoPedido','RejeicaoId','Data', 'StatusRejeicao']]



unificavalores = pd.merge(dflogpedidoatual,dfrejeicaowmsatual, on='CodigoPedido', how='left')
unificavalores
unificavalores['StatusRejeicao'].fillna('naoachou', inplace=True)
unificavalores
'''
unificavalores = pd.merge(dflogpedidoatual,dfrejeicaowmsatual, on='CodigoPedido', how='left')
unificavalores

In [ ]:

unificalocwms.loc[:,'REFERENCIA_PEDIDO_WMS']


unificalocwms.loc[:,'REFERENCIA_PEDIDO_WMS'] = unificalocwms.loc[:
    ,'REFERENCIA_PEDIDO_WMS'].apply(lambda k: str(k).replace('Ped.:','').split('-')[0].strip())


unificalocwms.to_excel("wmstesteexcel.xlsx")
unificavalores.loc[:,'CodigoPedido'] = unificavalores.loc[:,'CodigoPedido'].astype(str)
unificavalores.to_excel('wmsvalores.xlsx')



unificadf = pd.merge(unificavalores,unificalocwms, left_on='CodigoPedido'
    ,right_on='REFERENCIA_PEDIDO_WMS', how='left')

unificadf.loc[:,['CodigoPedido','REFERENCIA_PEDIDO_WMS']]
unificadf['REFERENCIA_PEDIDO_WMS'].fillna('naoachou', inplace=True)
unificadf
unificadf = unificadf.query("REFERENCIA_PEDIDO_WMS != 'naoachou'")

datas = unificadf.loc[:,['CodigoPedido','REFERENCIA_PEDIDO_WMS','NomeEtapa','datainserido','StatusRejeicao']]

datas

